<a href="https://colab.research.google.com/github/Elwing-Chou/tibame1224/blob/main/0120_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# demo一下requests函式庫
import requests
import bs4 as bs
url = "https://www.ptt.cc/bbs/Beauty/index3950.html"
response = requests.get(url)
print(response.status_code)
# 你可以看他當初送出的headers
print(response.request.headers)
print(response.headers)
# !!!!!!!!!!!!!! 記得要用的時候要.text
bs.BeautifulSoup(response.text)

In [ ]:
import json
import requests
import pandas as pd

url = "https://www.thsrc.com.tw/TimeTable/Search"
data = {
    "SearchType": "S",
    "Lang": "TW",
    "StartStation": "NanGang",
    "EndStation": "ZuoYing",
    "OutWardSearchDate": "2025/01/20",
    "OutWardSearchTime": "11:30",
    "ReturnSearchDate": "2025/01/20",
    "ReturnSearchTime": "11:30",
    "DiscountType":"",
}
h = {
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"
}
response = requests.post(url, data, headers=h)
# print(response.status_code)
result = json.loads(response.text)
# result->dict result["data"]->dict result["data"]["depature..."]->dict
data = result["data"]["DepartureTable"]
train_list = data["TrainItem"]
# train->dict
for train in train_list:
    train_number = train["TrainNumber"]
    train_depart_time = train["DepartureTime"]
    train_dest_time = train["DestinationTime"]
    train_duration = train["Duration"]
    print(train_number,
       train_depart_time,
       train_dest_time,
       train_duration)

df = pd.json_normalize(train_list)
df.to_csv("0120.csv", encoding="utf-8")

In [ ]:
# selenium: 如果你被擋住(妳是自動軟體.....)
# undeteced-chromedriver: https://github.com/ultrafunkamsterdam/undetected-chromedriver/tree/master
# 1. selenium 2. webdriver-manager
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import undetected_chromedriver as uc

if __name__ == '__main__':
    driver = uc.Chrome(use_subprocess=False)

    driver.get("https://www.google.com")
    driver.maximize_window()

    e = driver.find_element(By.CLASS_NAME, "gLFyf")
    e.send_keys("農曆新年")
    e.send_keys(Keys.ENTER)

    time.sleep(3)
    driver.quit()

```python
妳對應錯誤應該有兩種方法
1. 事前預防: if-else
例子(bs)
if xxxx == None:
    xxx
else:
    xxx.text

2. 事後解決: 準備好一個措施來應對隨時會發生的錯誤

try:
    xxx
    xxx
    xxx
except 某個錯誤:
    措施

a. 沒有發生錯誤: 完全不會跑到except
b. 發生錯誤(妳有except): 中斷try. 去執行except措施
c. 發生錯誤(妳沒有except): 中斷, 紅字

```

In [ ]:
# 1. selenium 2. webdriver-manager
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

# 如果不做設置, chrome視窗有可能會自動關閉
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options,
                          service=Service(ChromeDriverManager().install()))

driver.get("https://ani.gamer.com.tw/")
driver.maximize_window()

# click() 點擊
# send_keys(參數) 輸入
# selenium 和 beautifulsoup的差別
# bs: find           find_all
# se: find_element   find_elements
e = driver.find_element(By.ID, "anime-search-sky")
e.send_keys("鬼滅之刃")
time.sleep(1)
e.send_keys(Keys.ENTER)

# 找到他的搜尋結果
es = driver.find_elements(By.CLASS_NAME, "theme-list-main")
for e in es:
    try:
        # bs 和 se 的比較
        # bs: ["href"] .text
        # se: get_attribute("href")  .text
        link = e.get_attribute("href")
        ttime = e.find_element(By.CLASS_NAME, "theme-time")
        ttime_text = ttime.text
        tname = e.find_element(By.CLASS_NAME, "theme-name")
        tname_text = tname.text
        print(tname_text, ttime_text)
        print(link)
        print("-" * 30)
    except NoSuchElementException:
        print("pass")
        print("-" * 30)

time.sleep(1)
driver.close()